In [25]:
#Function to get isolines from the Here API. 

import requests
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Polygon
import folium
import citydelconfig as config
import pandas as pd


name = "hereapi"
ID = config.here_api_key
code = config.here_api_code

head = 'https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json?'
URL_Base = '{}app_id={}&app_code={}&mode=fastest;{};traffic:{}&start=geo!{},{}&range={}&rangetype={}'
URL_Base_dep = '{}app_id={}&app_code={}&mode=fastest;{};traffic:{}&departure={}&start=geo!{},{}&range={}&rangetype={}'

def get_isodata(location, type_iso = 'time', range_iso = 3600, travel_mode = 'car', traffic='disabled', departure=None): #receives a single location
    """ Function uses Here API to generate isolines depending on input parameters
    
    Parameters:
    location - Tuple of coordinates (lat, long)
    type_iso - Type of isoline, distance or time based, defaults to time
    range_iso - Range of the isoline depending on type_iso (in seconds), defaults to 1 hour
    travel_mode - Type of transit, can be truck, car or pedestrian
    traffic - toggles enabled or disabled
    departure - Departure time in format YYYY-MM-DDTHH:MM:SS (can include timezone)
    """
    
    if departure == None:
        url = URL_Base.format(head, ID, CODE, travel_mode, traffic, location[0], location[1], range_iso, type_iso) 
    else: 
        url = URL_Base_dep.format(head, ID, CODE, travel_mode, traffic, departure, location[0], location[1], range_iso, type_iso)
    try: 
        js = requests.get(url).json()['response']
        iso = js['isoline']
        coords = Polygon([(float(x.split(',')[1]), float(x.split(',')[0])) for x in iso[0]['component'][0]['shape']])
        geojs = gpd.GeoSeries([coords])
        geojs.crs = {'init' : 'epsg:4326'}
        return geojs
    except KeyError:
        js = requests.get(url).json()
        print(js)
        raise ValueError("HereAPI doesn't have data requested")
    except IndexError:
        raise ValueError("HereAPI doesn't have quality data")
        
            

In [26]:
#Function to plot isoline on a map, needs coordinates to start out

def isoplot(center, locations, outputfile = 'out.html'):
    """ Function creates a plot of the isoline on top of a map
    
    Parameters:
    center - Center for the map to be created (lat, long)
    locations - Geoseries/Geodataframe of isoline/isolines
    outputfile - Output location in .html extension
    """
    
    fm = folium.Map(location = center, zoom_start=10, tiles='CartoDBPositron')
    geojson = locations.__geo_interface__
    geojson['style'] = {"__comment": "all SVG styles allowed", "fill":"red", "stroke-width":"3", "fill-opacity":0.6}
    folium.GeoJson(geojson).add_to(fm)
    fm.save(outputfile)
    print('map saved to %s' %outputfile)

In [27]:
#secondary functions for different types of input data to plot
    
def isoplot_contour(center, locations, outputfile):
    #takes a center, and a list of isolines to create a contour map
    fm = folium.Map(location = center, zoom_start = 10, tiles = 'CartoDBPositron')
    for loc in locations:
        geojson = loc.__geo_interface__
        geojson['style'] = {"__comment": "all SVG styles allowed", "fill":"red", "stroke-width":"3", "fill-opacity":0.6}
        folium.GeoJson(geojson).add_to(fm)
    fm.save(outputfile)
    print('map saved to %s' %outputfile)
    
def get_isodata_list(locations, range_iso = 3600, type_iso = 'time', traffic = 'disabled'): 
    #recevies many locations, as a list of tuples of locations
    isoclines = []
    for loc in locations: 
        try:
            line = get_isodata(loc, range_iso, type_iso, traffic)
            isoclines.append(line)
        except ValueError:
            pass
    return isoclines

        
def make_list(centers, names, color, range_iso = 3600, type_iso = 'time', traffic = 'disabled'):
    for i, center in enumerate(centers):
        try:
            line = get_isodata(center, range_iso, type_iso, traffic)    
        except ValueError:
            pass
        isoplot(center, line, 'red', 'isoclines/' + names[i] +'.html')

In [30]:
#examples of using get_isodata and isoplot for different cities and times

abad_center = (23.027496, 72.572697)
locsabad = get_isodata(abad_center, range_iso = 3600, type_iso = 'time')
isoplot(abad_center, locsabad,'abad.html')
locsabad2 = get_isodata(abad_center, traffic='enabled', departure='2019-02-13T17:00:00')
isoplot(abad_center, locsabad2, '../abad_traffic.html')

mumbai_center = (19.060828, 72.872478)
locs = get_isodata(mumbai_center, traffic='enabled', departure='2019-02-13T17:00:00')
isoplot(mumbai_center, locs, 'mumbai_traffic.html')
locs2 = get_isodata(mumbai_center, traffic='disabled', departure = '2019-02-13T23:00:00')
isoplot(mumbai_center, locs2, 'mumbai_notraffic2.html')

new_york_center = (40.754177, -73.984632)
locsny = get_isodata(new_york_center, traffic='enabled', departure = '2019-02-13T17:00:00')
isoplot(new_york_center, locsny, 'ny_traffic.html')



#hereapi.isoplot(home, locs,'red','out.html')

map saved to abad.html
map saved to ../abad_traffic.html
map saved to mumbai_traffic.html
map saved to mumbai_notraffic2.html
map saved to ny_traffic.html


In [31]:
#Returns area of a geoseries in square kilometers
def geoseries_area(geoser):
    area_ser = geoser.to_crs({'proj':'cea'})
    return (area_ser.area[0] / 10**6)

In [32]:
#Reads in US cities shapefiles from cbsa

from geopandas import GeoSeries
us_shp_fp = "tl_2017_us_cbsa/tl_2017_us_cbsa.shp"
cbsa_data = gpd.read_file(us_shp_fp)

#Reads in US_counties shapefiles
US_counties_name = 'UScounties/UScounties.shp'
US_counties = gpd.read_file(US_counties_name)
US_counties.crs = {'init' : 'epsg:4326'} 





In [33]:
#Reads in Oecd Functional urban areas and the function returns the shapefile for a desired
#city by fuaname 

chicago_center = (41.8781136, -87.6297982)
us_shp_fp = "United States/United States.shp"
oecd_data = gpd.read_file(us_shp_fp)


def return_cityshp_oecd(city_name):
    shp =  oecd_data[oecd_data.fuaname == city_name]
    shp = shp.reset_index()
    geoshp = shp.geometry
    geoshp.crs = {'init' :'epsg:4326'}
    return geoshp
        

chicago_shp = return_cityshp_oecd('Chicago')

In [34]:
#Returns the marginal percentage areas that each isoline covers a city shapefile until the
#isoline completely engulfs the city shapefile. Each iteration increases isoline by 10 minutes. 

def cumulative_areas(center, oecd_shape):
    used_shape = oecd_shape.copy()
    oecd_area = geoseries_area(oecd_shape)
    overlap = 10000
    time = 600
    data_list = []
    locs_all = []
    cumu_area = overlap/oecd_area 
    while (cumu_area > 0):
        isoline = get_isodata(center, range_iso = time, traffic = 'enabled', departure = '2019-02-13T16:00:00')
        overlap = geoseries_area(isoline.intersection(used_shape))
        cumu_area = overlap/oecd_area
        data_list.append((time/60, cumu_area))
        used_shape = used_shape.difference(isoline)
        time += 600
    return data_list

In [35]:
#Plots data produced by cumulative_areas function
import numpy as np
import matplotlib.pyplot as plt

def plot_cumu_data(data):
    plt.plot([i[0] for i in data], np.cumsum([i[2] for i in data]), c='blue')
    


In [36]:
#Returns the intersection of the biggest isoline produced by the cumulative_areas function

def intersection_union(center, cum_areas, oecd_shape):
    time,v = cum_areas[-1]
    time = int(time) * 60
    isoline = get_isodata(center, time, traffic = 'enabled', departure = '2019-02-13T16:00:00')
    intersection = geoseries_area(isoline.intersection(oecd_shape))
    union = geoseries_area(isoline.union(oecd_shape))
    return intersection/union



In [37]:
#Returns cumulative_areas data and intersection over union data for a selection of metros
#area larger than 15037776415

def metros_cumu_areas(cbsa_data): 
    is_metro = cbsa_data['LSAD'] == 'M1'
    metros = cbsa_data[is_metro]
    is_big = metros['ALAND'] > 15037776415
    big_metros = metros[is_big]
    metros = big_metros.reset_index()
    CBSAFP = []
    names = []
    coords = []
    shapefiles = []
    cumu_areas = []
    iou_biggest = []
    for i, row in metros.iterrows():
        names.append(row['NAME'])
        CBSAFP.append(row['CBSAFP'])
        coords.append((float(row['INTPTLAT']), float(row['INTPTLON'])))
        geoshp = GeoSeries(row['geometry'])
        geoshp.crs = {'init' :'epsg:4326'}
        shapefiles.append(geoshp)
        try:
            cumu_area = cumulative_areas(coords[i], shapefiles[i])
            cumu_areas.append(cumu_area)
            iou_biggest.append(intersection_union(coords[i], cumu_area, shapefiles[i]))
        except ValueError:
            cumu_area = [(0,0)]
            cumu_areas.append(cumu_area)
            iou_biggest.append(0.0)
    return (cumu_areas, iou_biggest)

In [5]:
#Returns a data frame of data produced by function metros_cumu_areas and produces csv files

def cumu_areas_to_df(cumu_areas, iou_biggest)
    IOU_d = {'CBSAFP': CBSAFP, 'Names' : names, 'IOU' : iou_biggest}
    IOU_data = pd.DataFrame(data=IOU_d)
    CBSAFP_cumu = []
    names_cumu = []
    cumu_times_opened = []
    cumu_percents_opened = []
    for i, table in enumerate(cumu_areas):
        for time, percent in table:
            CBSAFP_cumu.append(CBSAFP[i])
            names_cumu.append(names[i])
            cumu_times_opened.append(time)
            cumu_percents_opened.append(percent)
    CA_d = {'CBSAFP' : CBSAFP_cumu, 'Names' : names_cumu, 
            'Isoline Time Buckets' : cumu_times_opened, 
            'Percentages' : cumu_percents_opened}
    CA_data = pd.DataFrame(data = CA_d)
    CA_data.to_csv('CA_data.csv')
    IOU_data.to_csv('IOU_data.csv')
    return IOU_data, CA_data

In [38]:
#Returns marginal population data from a merged data frame

pop_code = 'B01003_001E'

import censusdata
from census import Census

def marginal_population(merged_data):
    c = Census(census_API_key)
    data = []
    for a in range(16): #getvalue somehowelse
        data.append(0)
    state_data = []
    for state in merged_data.STATEFP.unique():
        state_data.extend(c.acs5.state_county_tract(pop_code, state, Census.ALL, Census.ALL))
    for index, d in enumerate(state_data):
        q = merged_data[(merged_data.STATEFP == d['state']) & 
                        (merged_data.COUNTYFP == d['county']) & 
                        (merged_data.TRACTCE == d['tract'])]
        print(q)
        if q.empty:
            continue
        else:
            try:
                data[q.index_right.item()] += d[pop_code]
            except KeyError:
                data[q.index_right.item()] = d[pop_code]
    return data

In [10]:
#Function returns a data frame consisting of the marginal population and areas of each
#isoline increasing in 10 minutes. The function is given co-ordinates of the center of cities
#and runs until it reaches 98% percentage of the city. 

import censusdata
from census import Census

bg_name = 'BG/tl_2018_{}_bg/tl_2018_{}_bg.shp'
pop_code = 'B01003_001E'
c = Census(config.census_api_key)

def isoline_pops(center, city_shape):
    used_shape = city_shape.copy()
    city_area = geoseries_area(city_shape)
    count = 0
    overlap = 10000
    time = 600
    times = []
    marginal_pops = []
    cumu_area = overlap/city_area 
    cumu_areas = []
    areas = []
    geometries = []
    merged = gpd.GeoDataFrame()
    states = []
    state_data = []
    states_read = dict()
    st_county_read = dict()
    while (sum(cumu_areas) < 0.98 and count<24): 
        isoline = get_isodata(center, range_iso = time, traffic = 'enabled', departure = '2019-02-13T16:00:00')
        if count==0:
            isoline_gdf = gpd.GeoDataFrame(geometry = isoline)
            isoline_gdf.crs = {'init':'epsg:4326'}
        else:
            isoline_gdf = gpd.GeoDataFrame(geometry = isoline.difference(prev_isoline))
            isoline_gdf.crs = {'init':'epsg:4326'}
        states = gpd.sjoin(US_counties, isoline_gdf, op="intersects").STATE_FIPS.unique()
        for state in states:
            if state in states_read.keys():
                pass
            else:
                bg_new = gpd.read_file(bg_name.format(state,state))
                bg_new.crs = ({'init':'epsg:4326'})
                states_read[state] = bg_new
                merged = pd.concat([merged, bg_new])
        marginal_pops.append(0)
        current_iso = gpd.sjoin(merged, isoline_gdf, op='intersects')
        for index, row in current_iso.iterrows():
            state_county = row['STATEFP'] + row['COUNTYFP']
            if not (state_county) in st_county_read.keys():
                st_county_read[state_county] = dict()
                county_bg_data = c.acs5.state_county_blockgroup(pop_code, row['STATEFP'], row['COUNTYFP'], Census.ALL)
                for d in county_bg_data:
                    st_county_read[state_county][state_county + d['tract'] + d['block group']] = d[pop_code]
            marginal_pops[count] += st_county_read[state_county][row['GEOID']]
            isoline = isoline.union(row['geometry'])
        overlap = geoseries_area(isoline.intersection(used_shape))
        geometries.append(isoline[0])
        times.append(time/60)
        areas.append(geoseries_area(isoline))
        cumu_area = overlap/city_area
        cumu_areas.append(cumu_area)
        used_shape = used_shape.difference(isoline)
        if count == 0:
            prev_isoline = gpd.GeoDataFrame(geometry = isoline)
            prev_isoline.crs = {'init' : 'epsg:4326'}
        else :
            prev_isoline = gpd.GeoDataFrame(geometry = prev_isoline.union(isoline))
            prev_isoline.crs = {'init' : 'epsg:4326'}
        time += 600
        count += 1
        print(count, cumu_area)
    optima = []
    for index, num in enumerate(marginal_pops):
        try:
            if num > marginal_pops[index+1] and num > marginal_pops[index-1]:
                optima.append(1)
            else:
                optima.append(0)
        except IndexError:
            optima.append(0)
    return_dict = {'Times' : times, 
                   'Cumulative Areas' : cumu_areas, 
                   'Areas' : areas,
                   'Marginal Population' : marginal_pops,
                  'Optima' : optima}
    return_frame = pd.DataFrame(data = return_dict)
    return return_frame
                

In [24]:
#Takes in list of cities with index, city name, state abbreviation as well as CBSA shapefiles
#and returns a dataframe of each cities with their marginal populations, cumulative areas, 
#local optima 

import re
google_api = "https://maps.googleapis.com/maps/api/geocode/json?address={}+{}&key=" + config.google_api_key
with open('top_cities.txt', 'r') as f:
        cities = f.readlines()

def top_30_pop_data(cities, cbsa_data):
    car_data_30 = pd.DataFrame()
    for row in cities:
        metro = row.split(',')
        city = metro[1]
        state = metro[2]
        print(state)
        is_metro = cbsa_data['LSAD'] == 'M1'
        metro_data = cbsa_data[is_metro]
        metro_data = metro_data[metro_data['NAME'].str.contains(city)]
        if len(metro_data) != 1:
            shp = metro_data[metro_data['NAME'].str.contains(state)]
        else:
            shp = metro_data[metro_data['NAME'].str.contains(city)]
        shp = shp.reset_index()
        geoshp = GeoSeries(shp['geometry'])
        geoshp.crs = {'init' : 'epsg:4326'}
        city = re.sub(r'\s+', '+', city)
        city = city.replace('.', '')
        url = google_api.format(city, state)
        js = requests.get(url).json()['results'][0]['geometry']
        lat = js['location']['lat']
        long = js['location']['lng']
        coords = (float(lat), float(long))
        print(city, state)
        print(coords)
        try:
            pop_data = isoline_pops(coords, geoshp)
            cbs_list = []
            name_list = []
            for a in range(0, len(pop_data)):
                cbs_list.append(shp['CBSAFP'][0])
                name_list.append(shp['NAME'][0])
            pop_data.insert(loc = 0, column = 'CBSAFP', value=cbs_list)
            pop_data.insert(loc = 1, column = 'NAME', value=name_list)
            car_data_30 = pd.concat([car_data_30, pop_data], sort=False)
            car_data_30.to_csv('car_data_30.csv')
        except (KeyboardInterrupt, SystemExit) as r:
            raise
        except Exception as e:
            print('error', e)
            pass

 